In [ ]:
from lxml import html
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


def region_car_url(region):

    url = "https://www.leboncoin.fr/voitures/offres/" + \ region + "/?th=1&parrot=0&brd=Renault&mdl=Zoe"
    text_lb = requests.get(url)
    soup = BeautifulSoup(text_lb.text,  'html.parser')

# On identifie la dernieère page

    der_page = list(map(lambda x: x['href'], soup.find_all(class_='element page static')))

    if len(der_page) != 0:
        idx_der_page = der_page[0].index("=")
        nb_der_page = int(der_page[0][idx_der_page + 1])
    else:
    nb_der_page = 1

    list_url_zoe = list()
    for i in range(1, nb_der_page + 1):
            url_i = "https://www.leboncoin.fr/voitures/offres/" + \ region + "/?th=1&parrot=" + str(i) + "&brd=Renault&mdl=Zoe"
    text_lb_i = requests.get(url_i)
    soup = BeautifulSoup(text_lb_i.text,  'html.parser')
    ml_url = map(lambda x: x['href'], soup.find_all(class_='list_item clearfix trackable'))
    list_url_zoe.extend(ml_url)
    
    return list_url_zoe


def extraction_information(url):

    text_page = requests.get("https:" + url)
    soup_page = BeautifulSoup(text_page.text, 'html.parser')
    title = soup_page.find(class_="no-border").text.replace("\n", "").replace("\t", "")
    clear_title = [i.lower() for i in title.split(" ") if i != " "]
    
    
    liste_name_modele = ['life', 'intens', 'zen']
    modele = [i for i in clear_title if i in liste_name_modele]

    type_car = ""
    # définition du type de voiture
    for idx, i in enumerate(clear_title):
        if i == 'charge':
            if clear_title[idx + 2] == "type":
                type_car = clear_title[idx] + " " + clear_title[idx + 1] + " " + clear_title[idx + 2] \ + " " + clear_title[idx + 3]
            else:
                type_car = clear_title[idx] + " " + clear_title[idx + 1]
            break

        elif i == 'type':
            type_car = clear_title[idx] + " " + clear_title[idx + 1]
            break
        modele = ''.join(modele)
   
    price = soup_page.find_all(class_='value')[0].text.replace("\xa0", "").replace("\n", "").replace(" ", "").replace("€","")
    year = soup_page.find_all(class_='value')[4].text.replace("\xa0", "").replace("\n", "").replace(" ", "")
    distance = soup_page.find_all(class_='value')[5].text.replace("\xa0", "").replace("\n", "").replace(" ", "").replace("KM","")

    # Le vendeur est il un professionnel
    
    pro = soup_page.find_all(class_='ispro')
    if len(pro) == 0:
        ispro = 'Particulier'
    else:
        ispro = soup_page.find_all(class_='ispro')[0].text.split(" ")[0]

  
    idx_description = len(soup_page.find_all(class_='value'))
    description = soup_page.find_all(class_='value')[idx_description - 1].text
    pattern_regex = ("(0|\\+33|0033)[1-9]([\S  \s -,.][0-9]{1,2}){4}")
    find_patern = re.compile(pattern_regex).search(description.strip())

    if find_patern is None:
        mobile_number = 'None'
    else:
        mobile_number = find_patern.group()

    value_return = {'Modele': modele, 'type': type_car, 'Année': year, 'Km': distance, 'Vendeur': ispro,'Prix €': price, 'Numéro': mobile_number}

    return value_return



def transform_data(ma_table) :
    for i in range(ma_table.shape[0]) :

        if ma_table.loc[i,"type"]!="" :

            type_c = ma_table.loc[i,"type"].split(" ")
            type_url = "+".join(type_c)
            url = "http://www.lacentrale.fr/cote-auto-renault-zoe-"+ ma_table.loc[i,"Modele"] + "+" \ +type_url+"-"+ ma_table.loc[i,"Année"] +".html"
        else :
            url = "http://www.lacentrale.fr/cote-auto-renault-zoe-"+ ma_table.loc[i,"Modele"]  \ +"-"+ ma_table.loc[i,"Année"] +".html"

        text_lc = requests.get(url)
        soup_lc = BeautifulSoup(text_lc.text,  'html.parser')

 
        price_init = soup_lc.find(class_="f24 bGrey9L txtRed pL15 mL15")

        if price_init is not None:
            price_lc = soup_lc.find(class_="f24 bGrey9L txtRed pL15 mL15").text.replace("\n","") \ .replace(" ","").replace("€","")
        else : 
            price_lc = np.nan
        ma_table.loc[i,"Price Central €"] = price_lc

    return ma_table 




def fullfil():
    # crée la table de remplissage
    columns_name = ['Modele', 'type', 'Année', 'Km', 'Vendeur', 'Prix €', 'Numéro']
    data_zoe = pd.DataFrame(colml_urlumns=columns_name)

    # On remplit pour chaque region
    for region in ['ile_de_france', 'aquitaine', 'provence_alpes_cote_d_azur']:

        liste_car = region_car_url(region)

        for v in liste_car:
            it_dict = extraction_information(v)
            data_zoe = data_zoe.append(it_dict, ignore_index=True)

    jmh_zoe = transform_data(data_zoe)

    jmh_zoe["Prix €"] = jmh_zoe["Prix €"].apply(float)
    jmh_zoe["Km"] = jmh_zoe["Km"].apply(float)
    jmh_zoe["Price Central €"] = jmh_zoe["Price Central €"].apply(float)

    jmh_zoe["Value Argus"] = jmh_zoe["Prix €"] - jmh_zoe["Price Central €"]

    jmh_zoe["Numéro"] = jmh_zoe["Numéro"].apply(lambda x : x.replace(".","").replace("/","").replace(" ",""))
    jmh_zoe["Numéro"] = jmh_zoe["Numéro"].apply(lambda x : None if len(x)!=10  else x) 


    return jmh_zoe

# execution du code

ma_table = fullfil()
path = "/Users/havard-macpro/Documents/Telecom_paristech/cours/INFMDI721 - kit big data/exercices"
ma_table.to_csv(path + "export_le_boncoin.csv")


